# Imports

In [ ]:
%matplotlib inline

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import warnings

warnings.filterwarnings('ignore')

# Patrón

Se utilizará el patrón de tablero de ajedrez de **9X6** de **opencv.org**.

In [ ]:
pattern = cv.imread("pattern.png")
plt.imshow(pattern)
plt.show()

# Fotos
Se utilizarán las siguientes fotos para realiza la calibración.

In [ ]:
filenames = glob.glob("./pictures/*")

for filename in filenames:
    print("processing image {0}...".format(filename))
    image = cv.imread(filename)
    plt.figure(filename)
    plt.imshow(image)
    plt.show()

# Prueba de flags para el método _findChessboardCorners_
Para esta prueba se utilizará una sola imagen.

In [ ]:
chessboard_size = (6, 9)
flag_names = ("cv.CALIB_CB_ADAPTIVE_THRESH", "cv.CALIB_CB_FILTER_QUADS",
              "cv.CALIB_CB_NORMALIZE_IMAGE", "cv.CALIB_CB_FAST_CHECK")

for flag_name in flag_names:
    print("Testing flag {0}...".format(flag_name))
    flag = eval(flag_name)
    original_image = cv.imread("pictures/05-center.jpg")
    grayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(grayscale_image, chessboard_size, flags=flag)
    if ret:
        print("Results for flag {0}: ".format(flag_name))
        print(corners)
        cv.drawChessboardCorners(original_image, chessboard_size, corners, ret)
        plt.figure(flag_name)
        plt.imshow(original_image)
        plt.show()
    else:
        print("No results for flag {0}: ".format(flag_name))

Se observó que el flag **CALIB_CB_NORMALIZE_IMAGE** no devuelve resultados. Para los otros 3 flags, no se observan diferencias a simple vista lo que se corrobora al comparar los puntos encontrados.

# Prueba de iteraciones para el método _cornerSubPix_
Para esta prueba se utilizará una sola imagen.


In [ ]:
iterations = (5, 10, 15, 20, 25)

for iteration in iterations:
    print("Testing {0} iterations...".format(iteration))
    original_image = cv.imread("pictures/05-center.jpg")
    grayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(grayscale_image, chessboard_size, flags=cv.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        print("Results for {0} iterations: ".format(iteration))
        print(corners)
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, iteration, 0.001)
        corners_subpix = cv.cornerSubPix(grayscale_image, corners, (5, 5), (-1, -1), criteria)
        cv.drawChessboardCorners(original_image, chessboard_size, corners_subpix, ret)
        plt.figure()
        plt.imshow(cv.cvtColor(original_image, cv.COLOR_BGR2RGB))
        plt.show()

No se observan diferencias a simple vista lo que se corrobora al comparar los puntos encontrados. Se utilizarán **5** iteraciones.

# Identificación de puntos imagen y puntos objeto

In [ ]:
max_iterations = 5
epsilon = 0.001
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, max_iterations, epsilon)
world_points = list()
image_points = list()
chessborard_world_points = np.zeros((np.prod(chessboard_size), 3), dtype=np.float32)
chessborard_world_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

for filename in filenames:
    print("processing image {0}...".format(filename))
    original_image = cv.imread(filename)
    grayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(grayscale_image, chessboard_size, flags=cv.CALIB_CB_ADAPTIVE_THRESH)
    if ret:
        world_points.append(chessborard_world_points)
        corners_subpix = cv.cornerSubPix(grayscale_image, corners, (5, 5), (-1, -1), criteria)
        image_points.append(corners_subpix)
        cv.drawChessboardCorners(original_image, chessboard_size, corners_subpix, ret)
        plt.figure()
        plt.imshow(cv.cvtColor(original_image, cv.COLOR_BGR2RGB))
        plt.show()

Se identificaron correctamente los puntos del tablero de ajedrez, ahora se procederá a la calibración de la cámara.

# Calibración de cámara

In [ ]:
image = cv.imread(filenames[0])
grayscale_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
image_height, image_width = grayscale_image.shape
ret, camera_matrix, distortion_coefficients, rvecs, tvecs = cv.calibrateCamera(
    world_points, image_points,(image_width, image_height), None, None)

print("Camera Matrix: \n{0}".format(camera_matrix))
print("Distortion Coefficients: \n{0}".format(distortion_coefficients))

# Rectificación de imágenes

In [ ]:
for filename in filenames:
    print("processing image {0}...".format(filename))
    image = cv.imread(filename)
    undistorted_image = cv.undistort(image, camera_matrix, distortion_coefficients)
    plt.figure()
    plt.imshow(undistorted_image)
    plt.show()

# Chequeo de los resultados
Se dibujará un primsma que abarque el tablero de ajedrez para verificar que los resultados son correctos.

In [ ]:
world_points = np.float32([[6, -1, 0], [6, 9, 0], [-1, 9, 0], [-1, -1, 0],
                        [6, -1, -1], [6, 9, -1], [-1, 9, -1], [-1, -1, -1]])
color = (0, 0, 255)
line_width = 5


for i in range(len(filenames)):
    print("processing image {0}...".format(filenames[i]))
    image = cv.imread(filenames[i])
    image_points = cv.projectPoints(world_points, rvecs[i], tvecs[i], camera_matrix,
                                    distortion_coefficients)[0]
    image_points = image_points[:, 0, :]
    cv.line(image, tuple(image_points[0]), tuple(image_points[1]), color, line_width)
    cv.line(image, tuple(image_points[1]), tuple(image_points[2]), color, line_width)
    cv.line(image, tuple(image_points[2]), tuple(image_points[3]), color, line_width)
    cv.line(image, tuple(image_points[3]), tuple(image_points[0]), color, line_width)
    cv.line(image, tuple(image_points[0]), tuple(image_points[4]), color, line_width)
    cv.line(image, tuple(image_points[1]), tuple(image_points[5]), color, line_width)
    cv.line(image, tuple(image_points[2]), tuple(image_points[6]), color, line_width)
    cv.line(image, tuple(image_points[3]), tuple(image_points[7]), color, line_width)
    cv.line(image, tuple(image_points[4]), tuple(image_points[5]), color, line_width)
    cv.line(image, tuple(image_points[5]), tuple(image_points[6]), color, line_width)
    cv.line(image, tuple(image_points[6]), tuple(image_points[7]), color, line_width)
    cv.line(image, tuple(image_points[7]), tuple(image_points[4]), color, line_width)

    plt.figure()
    plt.imshow(image[..., ::-1])
    plt.show()

Se verifica que se se dibuja el prisma correctamente.